# Homework №12

## Imports

In [53]:
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
# !pip install datasets tokenizers wandb xformers sentencepiece zstandard jsonlines accelerate -q
# !pip install -q bitsandbytes datasets accelerate loralib
!pip install numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.


In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import datasets
from datasets import load_dataset

import bitsandbytes as bnb
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model

import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Load model

In [4]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [5]:
model_name = 'lightblue/suzume-llama-3-8B-multilingual'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    cache_dir='./models'
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Prepare data

In [7]:
data = load_dataset('IlyaGusev/ru_turbo_alpaca', split='train')
data = data.filter(lambda x: x['label'] != 'bad_output')
data

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/29822 [00:00<?, ? examples/s]

Filter:   0%|          | 0/29822 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
    num_rows: 29132
})

In [24]:
df = pd.DataFrame(data)
df['text'] = df[['instruction', 'output']].apply(lambda x: '\n'.join(x), axis=1)
df = df[['text']]
df.head()

,text
0,Опиши процесс изготовления торта. Укажи ингред...
1,Напиши аналог данной пословицы на современный ...
2,"Представьте, что вы занимаетесь дизайном главн..."
3,Напиши рассказ о своем детстве и самом смешном...
4,Напишите краткую историю о двух лучших друзьях...


In [26]:
texts = datasets.Dataset.from_pandas(df)
texts

Dataset({
    features: ['text'],
    num_rows: 29132
})

In [69]:
MAX_LEN = 128
ds = texts.map(lambda samples: tokenizer(
    samples['text'],
    padding='max_length',
    truncation='only_first',
    max_length=MAX_LEN,
    ), batched=True)
ds = ds.remove_columns(['text', ])

Map:   0%|          | 0/29132 [00:00<?, ? examples/s]

## Fine-tuning

In [11]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [12]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [16]:
config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'o_proj', 'gate_proj', 'down_proj'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32,

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 65011712 || all params: 8095272960 || trainable%: 0.8030823953834906


### Обучение

In [17]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=ds,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

max_steps is given, it will override any value given in num_train_epochs


In [57]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

У меня так и не получилось решить проблему выше. Кажется, что-то идет не так в коллаторе, но я не понимаю, что именно (обычно у меня все работало + здесь же все то же самое, чтобы было в семинарской тетрадке). За несколько часов у меня не получилось найти ответ (уверена, что он какой-то очевидный, это всегда так работает), так что сдаю то, что есть.

В целом, все дальше должно работать, если заработает эта часть.

## Inference

In [ ]:
save_name = 'suzume_llama_lora'
model.save_pretrained(save_name)

In [ ]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             return_dict=True,
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            ).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to(device)
    output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

### Initial

In [ ]:
print(generate("Напиги рецепт печенья. ",  tokenizer, model))

In [ ]:
print(generate("Придумай историю про Машу и Дашу. ",  tokenizer, model))

### Perf

In [ ]:
model = PeftModel.from_pretrained(model, save_name)

In [ ]:
print(generate("Напиги рецепт печенья. ",  tokenizer, model))

In [ ]:
print(generate("Придумай историю про Машу и Дашу. ",  tokenizer, model))

Но ничего не работает...